<a href="https://colab.research.google.com/github/coryroyce/emnist_letter_exploration_and_prediction/blob/main/streamlit_application/Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit App - Letter Classification

Updated: 12/6/2021

Goal: Create an app wrapper for the letter prediction model. 

# Install Streamlit

In [17]:
!pip install streamlit -q
!pip install streamlit-drawable-canvas
!pip install ipykernel>=5.1.2
!pip install pydeck

# Clear output for this cell
from IPython.display import clear_output
clear_output()

Download Gihub repo to current Colab environment

In [1]:
!git clone https://github.com/coryroyce/emnist_letter_exploration_and_prediction

--2021-12-07 15:26:24--  https://github.com/coryroyce/emnist_letter_exploration_and_prediction/blob/main/streamlit_application/model_conv_v01.h5
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘model_conv_v01.h5’

model_conv_v01.h5       [ <=>                ] 164.15K  --.-KB/s    in 0.02s   

2021-12-07 15:26:25 (7.09 MB/s) - ‘model_conv_v01.h5’ saved [168088]



Create a app.py file for sreamlit to run

In [30]:
%%writefile app.py
import streamlit as st
from streamlit_drawable_canvas import st_canvas
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import pandas as pd
import plotly.express as px


# Cache model to load faster an only do it once rather than on every refresh
@st.cache(allow_output_mutation = True)
def load_models():
  # Load in the pre-trained model
  model_file_path = '/content/emnist_letter_exploration_and_prediction/streamlit_application/model_conv_v01.h5'
  model = tf.keras.models.load_model(model_file_path)
  return model

# Load the models
model = load_models()

# Define some valiables and helper functions
labels_caps = [
               'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
              ]
labels_dict_caps = {key: value for key, value in enumerate(labels_caps)}

# Define the Plotyly function
def create_probability_fig(model_prediction):
    df_temp = pd.DataFrame(model_prediction, columns=labels_dict_caps.values())
    df_temp = df_temp.transpose().reset_index()
    df_temp.columns = ['Label','Probability']
    fig = px.bar(df_temp, x='Label', y='Probability')
    return fig


# Create the Application
st.title('Letter and Didgit Prediction')

# Create a 2 column Streamlit layout
col1, col2 = st.columns(2)

with col1:
  st.markdown('Draw a capital letter here:')
  # Create a drawing canvas with desired properties
  canvas_result = st_canvas(
      fill_color="#ffffff",
      stroke_width=10,
      stroke_color='#ffffff',
      background_color="#000000",
      height=200,
      width=200,
      drawing_mode='freedraw',
      key="canvas",
  )

with col2:
  # Show that the resized image looks like
  st.markdown("What the model see's as input:")
  if canvas_result.image_data is not None:
      img = cv2.resize(canvas_result.image_data.astype('uint8'), (28, 28))
      img_rescaling = cv2.resize(img, (200, 200), interpolation=cv2.INTER_NEAREST)
      st.image(img_rescaling)

# Generate the prediciton based on the users drawings
if st.button('Predict'):
    x_user_input = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pred = model.predict(x_user_input.reshape(1, 28, 28, 1))
    pred_label = labels_dict_caps[pred.argmax()]
    st.write(f'Predicted Label: {pred_label}')

    # Create a plotly barchart of the predicted probablities
    fig = create_probability_fig(pred)
    st.plotly_chart(fig, use_container_width = True)


Overwriting app.py


Load an example app for reference

In [31]:
!streamlit run app.py & npx localtunnel --port 8501

npx: installed 22 in 2.813s
your url is: https://pretty-fireant-68.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.247.69.149:8501

2021-12-07 18:25:13.846588: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
  Stopping...
^C


# Reference

Method for insering a Drawable Canvas and have that converted back to a python ML model was inspired by bhupendrak9917's notebook [Github](https://github.com/bhupendrak9917/My-AI-Projects/tree/main/MNIST_Streamlit)